In [1]:
import warnings

import geopandas as gpd
import libpysal
import momepy
import osmnx
import os
import configparser
import numpy as np

#from bokeh.io import output_notebook
#output_notebook()

In [2]:
cities = ['Jerusalem', 'Tel Aviv', 'Haifa', 'Rishon LeZion', 'Petah Tikva', 'Ashdod', 'Netanya', 'Beer Sheva', 'Bnei Brak', 'Holon', 'Ramat Gan', 'Rehovot', 'Ashkelon', 'Bat Yam', 'Beit Shemesh', 'Kfar Saba', 'Herzliya', 'Hadera', 'Modiin', 'Nazareth']

# add CRS
cities = {k: "EPSG:2039" for k in cities}


In [3]:
# Read the configuration file
config = configparser.ConfigParser()
config_file = 'config.ini'

# Check if file exists
if not os.path.isfile(config_file):
    # Create a simple config file
    config['Paths'] = {
        'gdb_folder': '/path/to/your/gdb/folder',
        'output_dir': '/path/to/your/output/folder'
    }
    with open(config_file, 'w') as configfile:
        config.write(configfile)
    print(f"Created configuration file '{config_file}'.")

# Read the configuration file
config.read(config_file)
gdb_folder = config['Paths']['gdb_folder']
output_dir = config['Paths']['output_dir']

# Ensure the directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:

# Dictionaries to store data for each city
buildings_data = {}
streets_data = {}
tessellation_data = {}

# Loop through each city calculate or load buildings, streets and tessealtions
for city, local_crs in cities.items():
    print(f"Processing {city}")

    # File paths for the saved data
    buildings_file = os.path.join(output_dir, f"{city}_buildings.geojson")
    tessellation_file = os.path.join(output_dir, f"{city}_tessellation.geojson")
    streets_file = os.path.join(output_dir, f"{city}_streets.geojson")

    # Check if files already exist, if so, load them
    if os.path.exists(buildings_file) and os.path.exists(tessellation_file) and os.path.exists(streets_file):
        print(f"Loading saved data for {city}...")
        buildings = gpd.read_file(buildings_file)
        tessellation = gpd.read_file(tessellation_file)
        streets = gpd.read_file(streets_file)
    else:
        print(f"Fetching and processing data for {city}...")
        # Get buildings and street data
        buildings = osmnx.geometries.geometries_from_place(city, tags={'building': True})
        buildings = buildings[buildings.geom_type == "Polygon"].reset_index(drop=True)
        buildings = buildings[["geometry"]].to_crs(local_crs)
        buildings["uID"] = range(len(buildings))

        osm_graph = osmnx.graph_from_place(city, network_type='drive')
        osm_graph = osmnx.projection.project_graph(osm_graph, to_crs=local_crs)

        streets = osmnx.graph_to_gdfs(
            osm_graph,
            nodes=False,
            edges=True,
            node_geometry=False,
            fill_edge_geometry=True
        )
        streets = momepy.remove_false_nodes(streets)
        streets = streets[["geometry"]]
        streets["nID"] = range(len(streets))

        # Generate tessellation and perform metrics
        limit = momepy.buffered_limit(buildings, 100)
        tessellation = momepy.Tessellation(buildings, "uID", limit, verbose=False, segment=1)
        tessellation = tessellation.tessellation

        buildings = buildings.sjoin_nearest(streets, max_distance=1000, how="left")
        buildings = buildings.drop_duplicates("uID").drop(columns="index_right")
        tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='left')

        # Save the data for future use
        print(f"Saving data for {city}...")
        buildings.to_file(buildings_file, driver="GeoJSON")
        tessellation.to_file(tessellation_file, driver="GeoJSON")
        streets.to_file(streets_file, driver="GeoJSON")
    
    # Store the data in dictionaries
    buildings_data[city] = buildings
    streets_data[city] = streets
    tessellation_data[city] = tessellation

Processing Jerusalem
Loading saved data for Jerusalem...
Processing Tel Aviv
Loading saved data for Tel Aviv...
Processing Haifa
Loading saved data for Haifa...
Processing Rishon LeZion
Loading saved data for Rishon LeZion...
Processing Petah Tikva
Loading saved data for Petah Tikva...
Processing Ashdod
Loading saved data for Ashdod...
Processing Netanya
Loading saved data for Netanya...
Processing Beer Sheva
Loading saved data for Beer Sheva...
Processing Bnei Brak
Loading saved data for Bnei Brak...
Processing Holon
Loading saved data for Holon...
Processing Ramat Gan
Loading saved data for Ramat Gan...
Processing Rehovot
Loading saved data for Rehovot...
Processing Ashkelon
Loading saved data for Ashkelon...
Processing Bat Yam
Loading saved data for Bat Yam...
Processing Beit Shemesh
Loading saved data for Beit Shemesh...
Processing Kfar Saba
Loading saved data for Kfar Saba...
Processing Herzliya
Loading saved data for Herzliya...
Processing Hadera
Loading saved data for Hadera...


neighbors distance Q1

In [5]:
# create output directories
for dirname in ["neighbor_distance_Q1", "neighbor_distance_Q3", "shared_walls_length", "covered_area"]:
    dir_to_make = os.path.join(output_dir, dirname)
    if not os.path.exists(dir_to_make):
        os.makedirs(dir_to_make)

In [6]:
output_df_dir = os.path.join(output_dir, "neighbor_distance_Q1") # Replace with your desired folder path

# Dictionary to store the DataFrame for each city
city_metric_dfs = {}

# Loop through each city to calculate metrics based on stored data
for city in buildings_data.keys():
    print(f"Calculating metrics for {city}...")

    # Retrieve the data for the city from the dictionaries
    buildings = buildings_data[city]
    streets = streets_data[city]
    tessellation = tessellation_data[city]  
    
    queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tessellation, ids="uID", silence_warnings=True)
    tessellation["neighbors"] = momepy.Neighbors(tessellation, queen_1, "uID", weighted=True, verbose=False).series
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        buildings["neighbor_distance"] = momepy.NeighborDistance(buildings, queen_1, "uID", verbose=False).series

    buildings["log_neighbor_distance"] = np.log(buildings["neighbor_distance"] + 1) / np.log(10)

    # Create a DataFrame containing the necessary metrics for this city
    city_metrics_df = buildings[["uID", "neighbor_distance", "log_neighbor_distance"]].copy()

    # Store the DataFrame in the dictionary
    city_metric_dfs[city] = city_metrics_df

    # Display the DataFrame for the current city
    print(f"\nMetrics DataFrame for {city}:")
    print(city_metric_dfs[city].head())

    # Define the output file path using city name and metric
    output_file = os.path.join(output_df_dir, f"{city}_neighbor_distance_Q1.csv")

    # Save the DataFrame as a CSV file
    city_metrics_df.to_csv(output_file, index=False)

    print(f"Saved metrics DataFrame for {city} to {output_file}")


Calculating metrics for Jerusalem...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/1836647266.py:16: FutureWarning: Class based API like `momepy.Neighbors` is deprecated. Replace it with `momepy.neighbors` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  tessellation["neighbors"] = momepy.Neighbors(tessellation, queen_1, "uID", weighted=True, verbose=False).series



Metrics DataFrame for Jerusalem:
   uID  neighbor_distance  log_neighbor_distance
0    0          41.893135               1.632388
1    1          20.728500               1.337030
2    2         122.810058               2.092756
3    3           3.291272               0.632586
4    4           2.412582               0.533083
Saved metrics DataFrame for Jerusalem to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q1/Jerusalem_neighbor_distance_Q1.csv
Calculating metrics for Tel Aviv...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/1836647266.py:16: FutureWarning: Class based API like `momepy.Neighbors` is deprecated. Replace it with `momepy.neighbors` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  tessellation["neighbors"] = momepy.Neighbors(tessellation, queen_1, "uID", weighted=True, verbose=False).series



Metrics DataFrame for Tel Aviv:
   uID  neighbor_distance  log_neighbor_distance
0    0          47.506637               1.685801
1    1          67.795453               1.837560
2    2         119.657735               2.081555
3    3          32.044263               1.519096
4    4         144.011729               2.161403
Saved metrics DataFrame for Tel Aviv to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q1/Tel Aviv_neighbor_distance_Q1.csv
Calculating metrics for Haifa...


...


Metrics DataFrame for Hadera:
   uID  neighbor_distance  log_neighbor_distance
0    0          73.737704               1.873540
1    1         124.848059               2.099847
2    2         117.362208               2.073213
3    3         101.634512               2.011293
4    4          51.744698               1.722179
Saved metrics DataFrame for Hadera to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q1/Hadera_neighbor_distance_Q1.csv
Calculating metrics for Modiin...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/1836647266.py:16: FutureWarning: Class based API like `momepy.Neighbors` is deprecated. Replace it with `momepy.neighbors` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  tessellation["neighbors"] = momepy.Neighbors(tessellation, queen_1, "uID", weighted=True, verbose=False).series



Metrics DataFrame for Modiin:
   uID  neighbor_distance  log_neighbor_distance
0    0          39.405827               1.606444
1    1          51.689191               1.721722
2    2          78.361407               1.899609
3    3          92.713786               1.971803
4    4          31.552228               1.512581
Saved metrics DataFrame for Modiin to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q1/Modiin_neighbor_distance_Q1.csv
Calculating metrics for Nazareth...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/1836647266.py:16: FutureWarning: Class based API like `momepy.Neighbors` is deprecated. Replace it with `momepy.neighbors` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  tessellation["neighbors"] = momepy.Neighbors(tessellation, queen_1, "uID", weighted=True, verbose=False).series



Metrics DataFrame for Nazareth:
   uID  neighbor_distance  log_neighbor_distance
0    0          32.733206               1.528058
1    1          44.572060               1.658699
2    2          10.672604               1.067168
3    3          75.564280               1.884026
4    4          79.863505               1.907753
Saved metrics DataFrame for Nazareth to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q1/Nazareth_neighbor_distance_Q1.csv


neighbors distance Q3

In [7]:
output_df_dir = os.path.join(output_dir, "neighbor_distance_Q3")

# Dictionary to store the DataFrame for each city
city_metric_dfs = {}

# Loop through each city to calculate metrics based on stored data
for city in buildings_data.keys():
    print(f"Calculating metrics for {city}...")

    # Retrieve the data for the city from the dictionaries
    buildings = buildings_data[city]
    streets = streets_data[city]
    tessellation = tessellation_data[city]  
    
    queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tessellation, ids="uID", silence_warnings=True)
    # Create a higher-order spatial weights matrix for tessellation (considering neighbors up to 3rd order)
    queen_3 = momepy.sw_high(k=3, weights=queen_1)

    # Calculate neighbor distances using the higher-order spatial weights
    buildings['neighbor_distance_q3'] = momepy.NeighborDistance(buildings, queen_3, "uID", verbose=False).series

    # Create a DataFrame containing the necessary metrics for this city
    city_metrics_df = buildings[["uID", "neighbor_distance_q3"]].copy()

    # Store the DataFrame in the dictionary
    city_metric_dfs[city] = city_metrics_df

    # Display the DataFrame for the current city
    print(f"\nMetrics DataFrame for {city}:")
    print(city_metric_dfs[city].head())

    # Define the output file path using city name and metric
    output_file = os.path.join(output_df_dir, f"{city}_neighbor_distance_Q3.csv")

    # Save the DataFrame as a CSV file
    city_metrics_df.to_csv(output_file, index=False)

    print(f"Saved metrics DataFrame for {city} to {output_file}")


Calculating metrics for Jerusalem...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3993208919.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=3, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3993208919.py:20: FutureWarning: Class based API like `momepy.NeighborDistance` is deprecated. Replace it with `momepy.neighbor_distance` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  buildings['neighbor_distance_q3'] = momepy.NeighborDistance(buildings, queen_3, "uID", verbose=False).series



Metrics DataFrame for Jerusalem:
   uID  neighbor_distance_q3
0    0            121.909552
1    1            131.074745
2    2            225.911765
3    3             72.594325
4    4            121.442453
Saved metrics DataFrame for Jerusalem to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q3/Jerusalem_neighbor_distance_Q3.csv
Calculating metrics for Tel Aviv...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3993208919.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=3, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3993208919.py:20: FutureWarning: Class based API like `momepy.NeighborDistance` is deprecated. Replace it with `momepy.neighbor_distance` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  buildings['neighbor_distance_q3'] = momepy.NeighborDistance(buildings, queen_3, "uID", verbose=False).series



Metrics DataFrame for Tel Aviv:
   uID  neighbor_distance_q3
0    0            124.578091
1    1            201.610202
2    2            235.830444
3    3             92.354157
4    4            151.534608
Saved metrics DataFrame for Tel Aviv to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q3/Tel Aviv_neighbor_distance_Q3.csv
Calculating metrics for Haifa...


...


Metrics DataFrame for Hadera:
   uID  neighbor_distance_q3
0    0            149.369898
1    1            313.404131
2    2            274.377137
3    3            240.476803
4    4            142.981992
Saved metrics DataFrame for Hadera to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q3/Hadera_neighbor_distance_Q3.csv
Calculating metrics for Modiin...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3993208919.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=3, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3993208919.py:20: FutureWarning: Class based API like `momepy.NeighborDistance` is deprecated. Replace it with `momepy.neighbor_distance` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  buildings['neighbor_distance_q3'] = momepy.NeighborDistance(buildings, queen_3, "uID", verbose=False).series



Metrics DataFrame for Modiin:
   uID  neighbor_distance_q3
0    0            107.978099
1    1            238.005511
2    2            178.620772
3    3            217.765268
4    4            109.794765
Saved metrics DataFrame for Modiin to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q3/Modiin_neighbor_distance_Q3.csv
Calculating metrics for Nazareth...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3993208919.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=3, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3993208919.py:20: FutureWarning: Class based API like `momepy.NeighborDistance` is deprecated. Replace it with `momepy.neighbor_distance` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  buildings['neighbor_distance_q3'] = momepy.NeighborDistance(buildings, queen_3, "uID", verbose=False).series



Metrics DataFrame for Nazareth:
   uID  neighbor_distance_q3
0    0            124.077565
1    1             98.206381
2    2             96.020656
3    3            149.705602
4    4            162.737930
Saved metrics DataFrame for Nazareth to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/neighbor_distance_Q3/Nazareth_neighbor_distance_Q3.csv


shared_walls_length

In [8]:
output_df_dir = os.path.join(output_dir, "shared_walls_length")

# Dictionary to store the DataFrame for each city
city_metric_dfs = {}

# Loop through each city to calculate shared walls length based on stored data
for city in buildings_data.keys():
    print(f"Calculating shared walls length for {city}...")

    # Retrieve the data for the city from the dictionaries
    buildings = buildings_data[city]
    tessellation = tessellation_data[city]

    # Calculate shared walls length for each building
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        buildings["shared_walls_length"] = momepy.SharedWalls(buildings).series

    # Create a DataFrame containing the necessary metrics for this city
    city_metrics_df = buildings[["uID", "shared_walls_length"]].copy()

    # Store the DataFrame in the dictionary
    city_metric_dfs[city] = city_metrics_df

    # Display the DataFrame for the current city
    print(f"\nMetrics DataFrame for {city}:")
    print(city_metric_dfs[city].head())

    # Define the output file path using the city name and metric with '.pkl' extension
    output_file = os.path.join(output_df_dir, f"{city}_shared_walls_length.csv")

    # Save the DataFrame as a pickle file
    city_metrics_df.to_csv(output_file, index=False)


    print(f"Saved metrics DataFrame for {city} as pickle to {output_file}")

Calculating shared walls length for Jerusalem...

Metrics DataFrame for Jerusalem:
   uID  shared_walls_length
0    0        -2.842171e-14
1    1        -2.842171e-14
2    2        -5.684342e-14
3    3         2.842171e-14
4    4         4.884836e+01
Saved metrics DataFrame for Jerusalem as pickle to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/shared_walls_length/Jerusalem_shared_walls_length.csv
Calculating shared walls length for Tel Aviv...

Metrics DataFrame for Tel Aviv:
   uID  shared_walls_length
0    0         2.842171e-14
1    1         4.613901e+02
2    2        -2.842171e-14
3    3         0.000000e+00
4    4         0.000000e+00
Saved metrics DataFrame for Tel Aviv as pickle to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/shared_walls_length/Tel Aviv_shared_walls_length.csv
Calculating shared walls length for Haifa...

Metrics DataFrame for Haifa:
   uID  shared_walls_length
0    0             0.000000
1    1             0.000000
2    2           192.671951
3  

Covered Area

In [10]:
output_df_dir = os.path.join(output_dir, "covered_area")

# Dictionary to store the DataFrame for each city
city_metric_dfs = {}

# Loop through each city to calculate metrics based on stored data
for city in buildings_data.keys():
    print(f"Calculating metrics for {city}...")

    # Retrieve the data for the city from the dictionaries
    buildings = buildings_data[city]
    streets = streets_data[city]
    tessellation = tessellation_data[city]  
    
    queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tessellation, ids="uID", silence_warnings=True)
    # Create a higher-order spatial weights matrix for tessellation (considering neighbors up to k-th order)
    queen_3 = momepy.sw_high(k=1, weights=queen_1)

    # Calculate covered area
    tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_3, "uID", verbose=False).series

    # Create a DataFrame containing the necessary metrics for this city
    city_metrics_df = tessellation[["uID", "covered_area"]].copy()

    # Store the DataFrame in the dictionary
    city_metric_dfs[city] = city_metrics_df

    # Display the DataFrame for the current city
    print(f"\nMetrics DataFrame for {city}:")
    print(city_metric_dfs[city].head())

    # Define the output file path using city name and metric
    output_file = os.path.join(output_df_dir, f"{city}_covered_area.csv")

    # Save the DataFrame as a CSV file
    city_metrics_df.to_csv(output_file, index=False)

    print(f"Saved metrics DataFrame for {city} to {output_file}")


Calculating metrics for Jerusalem...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3086072035.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=1, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3086072035.py:20: FutureWarning: `momepy.CoveredArea` is deprecated. Replace it with `.describe()` method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_3, "uID", verbose=False).series



Metrics DataFrame for Jerusalem:
     uID  covered_area
0  32847  43491.590437
1  32755  49947.946601
2  32812  54584.962624
3  32801  46198.085379
4  32798  40259.372105
Saved metrics DataFrame for Jerusalem to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/covered_area/Jerusalem_covered_area.csv
Calculating metrics for Tel Aviv...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3086072035.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=1, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3086072035.py:20: FutureWarning: `momepy.CoveredArea` is deprecated. Replace it with `.describe()` method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_3, "uID", verbose=False).series



Metrics DataFrame for Tel Aviv:
     uID  covered_area
0  10274  34922.405235
1  24334  53985.153437
2  14014  43636.439811
3  13953  52091.710847
4  14010  49425.681480
Saved metrics DataFrame for Tel Aviv to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/covered_area/Tel Aviv_covered_area.csv
Calculating metrics for Haifa...


...


Metrics DataFrame for Hadera:
    uID  covered_area
0  1091  37706.250547
1  1493  15158.494329
2  1518  19629.195065
3  1511  18511.014085
4  1480  14350.273349
Saved metrics DataFrame for Hadera to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/covered_area/Hadera_covered_area.csv
Calculating metrics for Modiin...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3086072035.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=1, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3086072035.py:20: FutureWarning: `momepy.CoveredArea` is deprecated. Replace it with `.describe()` method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_3, "uID", verbose=False).series



Metrics DataFrame for Modiin:
    uID  covered_area
0  2921  40166.499311
1  1942  20445.721823
2  1938  23473.544276
3  1928  28714.985669
4  1937  23179.448360
Saved metrics DataFrame for Modiin to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/covered_area/Modiin_covered_area.csv
Calculating metrics for Nazareth...


/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3086072035.py:17: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  queen_3 = momepy.sw_high(k=1, weights=queen_1)
/var/folders/vq/5fctqqys06l8n2l1cjxhxvp80000gn/T/ipykernel_72660/3086072035.py:20: FutureWarning: `momepy.CoveredArea` is deprecated. Replace it with `.describe()` method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_3, "uID", verbose=False).series



Metrics DataFrame for Nazareth:
    uID  covered_area
0  3337  26871.876325
1  3338  47211.870970
2  3340   9029.561790
3  3341  28896.439774
4  3342  36028.746451
Saved metrics DataFrame for Nazareth to /Users/dpelleg/work/dssg/DSSG_mapping_Jerus/output/covered_area/Nazareth_covered_area.csv
